# comercio_ext_auxiliares.tb_auxiliar_paises
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/PAIS_delta`
## 📌 Descrição do arquivo
Referência de **Países** com códigos ISO e nomes multilíngues (PT/EN/ES).
|Coluna|Descrição|
|---|---|
|`CO_PAIS`|Código interno do país|
|`CO_PAIS_ISON3`|Código ISO numérico (3)|
|`CO_PAIS_ISOA3`|Código ISO alfabético (3)|
|`NO_PAIS`|Nome em PT|
|`NO_PAIS_ING`|Nome em EN|
|`NO_PAIS_ESP`|Nome em ES|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:
val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/PAIS_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_auxiliares/tb_auxiliar_paises/"   
val silverTable =    "tb_auxiliar_paises"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_PAIS" , StringType ,  nullable = false ) ,
      StructField ( "CO_PAIS_ISON3" , StringType ,  nullable = false ) ,
      StructField ( "CO_PAIS_ISOA3" , StringType ,  nullable = false ) ,
      StructField ( "NO_PAIS" , StringType ,  nullable = false ) ,
      StructField ( "NO_PAIS_ING" , StringType ,  nullable = false ) ,
      StructField ( "NO_PAIS_ESP" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_PAIS", trim(upper(col("CO_PAIS").cast(StringType))))
  .withColumn("CO_PAIS_ISON3", trim(upper(col("CO_PAIS_ISON3").cast(StringType))))
  .withColumn("CO_PAIS_ISOA3", trim(upper(col("CO_PAIS_ISOA3").cast(StringType))))
  .withColumn("NO_PAIS", col("NO_PAIS").cast(StringType))
  .withColumn("NO_PAIS_ING", col("NO_PAIS_ING").cast(StringType))
  .withColumn("NO_PAIS_ESP", col("NO_PAIS_ESP").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/PAIS_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_PAIS").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_PAIS"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_PAIS"), col("CO_PAIS_ISON3"), col("CO_PAIS_ISOA3"), col("NO_PAIS"), col("NO_PAIS_ING"), col("NO_PAIS_ESP"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}